# Getting Started with Prompt Engineering
Readapted from: https://github.com/dair-ai/Prompt-Engineering-Guide


This notebook contains examples and exercises to learning about prompt engineering.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics

Objectives
- Load the libraries
- Review the format
- Cover basic prompts
- Review common use cases

Below we are loading the necessary libraries, utilities, and configurations.

In [0]:
# update or install the necessary libraries
!pip install openai
!pip install --upgrade typing_extensions

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from openai import OpenAI
import os
import IPython

DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url="https://adb-3750392177977863.3.azuredatabricks.net/serving-endpoints"
)

In [0]:
chat_completion = client.chat.completions.create(
  messages=[
  {
    "role": "system",
    "content": "You are an AI assistant"
  },
  {
    "role": "user",
    "content": "Hi how are you doing?"
  }
  ],
  model="databricks-gpt-oss-120b",
  max_tokens=256
)

In [0]:
chat_completion.choices[0].message.content

[{'type': 'reasoning',
  'summary': [{'type': 'summary_text',
    'text': 'The user greeting. Just respond politely.'}]},
 {'type': 'text',
  'text': 'Hello! I’m doing great, thanks for asking. How can I help you today?'}]

In [0]:
def set_open_params(
    model="databricks-gpt-oss-120b",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = client.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

Basic prompt example:

In [0]:
# basic example
params = set_open_params()

prompt = "The sky is"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [0]:
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

The sky is a vast, ever‑changing canvas—deep sapphire in the early morning, brushed with the soft pinks and golds of sunrise, then brightening to a clear, crystal‑blue that stretches endlessly over the horizon. By midday it glows with a fierce, almost electric intensity, while wisps of white clouds drift lazily like cotton islands. As the sun dips toward the west, the sky transforms again, painted in molten oranges, fiery reds, and bruised purples, each hue melting into the next in a breathtaking display. When night finally falls, the sky becomes a velvet blanket studded with countless stars, a silent testament to the mysteries that lie far beyond our world.

Try with different temperature to compare results:

In [0]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

The sky is a constantly changing canvas—sometimes a clear, bright blue, other times painted with soft pinks and oranges at sunrise, deep purples and reds at sunset, or dramatic shades of gray when clouds gather. What aspect of the sky were you thinking about? 🌤️

### 1.1 Text Summarization

In [0]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

Antibiotics are drugs that kill or inhibit bacteria—taken orally or intravenously—to help the immune system clear infections, but they don’t work against viruses and misuse can cause antibiotic resistance.

Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [0]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])


Mice.

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

Exercise: Edit prompt and get the model to respond that it isn't sure about the answer. 

### 1.3 Text Classification

In [0]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

Neutral

Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected. 

### 1.4 Role Playing

In [0]:
params = set_open_params(max_tokens=256)
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific but short.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }

]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

Black holes form when mass is compressed within its Schwarzschild radius, \(R_s = \frac{2GM}{c^2}\). The main astrophysical channels are:

1. **Stellar collapse** – A massive star (>≈ 20 M☉) exhausts nuclear fuel, its core can no longer support itself against gravity, and it collapses directly into a black hole (or via a brief neutron‑star phase).

2. **Binary mergers** – Coalescence of two neutron stars or a neutron star and a black hole can produce a remnant whose mass exceeds the Tolman–Oppenheimer–Volkoff limit, yielding a black hole.

3. **Supermassive black holes** – Likely grow from seed black holes (∼10²–10⁵ M☉) formed by direct gas collapse in early protogalaxies, or

Exercise: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [0]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])


Here’s a simple MySQL query that returns every student who belongs to the **Computer Science** department.

```sql
SELECT
    s.StudentId,
    s.StudentName,
    s.DepartmentId
FROM
    students AS s
JOIN
    departments AS d
    ON s.DepartmentId = d.DepartmentId
WHERE
    d.DepartmentName = 'Computer Science';
```

### How it works
| Clause | Purpose |
|--------|---------|
| `FROM students AS s` | Starts with the `students` table (aliased as `s`). |
| `JOIN departments AS d ON s.DepartmentId = d.DepartmentId` | Links each student to its department via the foreign key `DepartmentId`. |
| `WHERE d.DepartmentName = 'Computer Science'` | Filters the result set to only those rows where the department’s name matches *Computer Science*. |
| `SELECT s.StudentId, s.StudentName, s.DepartmentId` | Returns the columns you likely need; you can add/remove columns as required (e.g., `d.DepartmentName`).

### 1.6 Reasoning

In [0]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [0]:
response.choices[0].message.content

[{'type': 'reasoning',
  'summary': [{'type': 'summary_text',
    'text': 'The user asks: "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.\n\nSolve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."\n\nWe need to list odd numbers: 15, 5, 13, 7, 1. Sum them: 15+5=20, +13=33, +7=40, +1=41. Wait check: 15+5=20, +13=33, +7=40, +1=41. That\'s odd, not even. But the problem statement says "The odd numbers in this group add up to an even number". Perhaps there is a mistake? Let\'s compute again: Did we miss any odd? 32 is even, 82 even. So odd numbers are indeed 15,5,13,7,1. Sum = 41 (odd). So the statement is false. But they ask to solve by breaking steps: identify odd numbers, add them, and indicate whether result is odd or even. So answer: odd numbers: 15,5'}]}]

Exercise: Improve the prompt to have a better structure and output format.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [0]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

A: The answer is False.

### 2.3 Chain-of-Thought (CoT) Prompting

In [0]:
params = set_open_params(max_tokens=512)
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

The odd numbers in the list are  

\[
15,\;5,\;13,\;7,\;1
\]

Adding them:

\[
15 + 5 = 20 \\
20 + 13 = 33 \\
33 + 7 = 40 \\
40 + 1 = 41
\]

The total is **41**, which is an odd number.  

Therefore, the statement “The odd numbers in this group add up to an even number” is **False**.

### 2.4 Zero-shot CoT

In [0]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content[1]['text'])

Let's break it down step by step:

1. **First purchase:**  
   - You bought **10 apples**.  
   - Apples you have now: **10**.

2. **Giving apples away:**  
   - Gave **2 apples** to the neighbor → 10 − 2 = **8**.  
   - Gave **2 apples** to the repairman → 8 − 2 = **6**.  

   Apples you have now: **6**.

3. **Second purchase:**  
   - Bought **5 more apples** → 6 + 5 = **11**.  

   Apples you have now: **11**.

4. **Eating an apple:**  
   - Ate **1 apple** → 11 − 1 = **10**.

**Result:** After all those transactions, you are left with **10 apples**.

---